In [ ]:
# | default_exp article

In [ ]:
# | export
from sqlmodel import SQLModel, Field, create_engine, Session, select, Column
from sqlalchemy import JSON
from pydantic import field_validator
from datetime import datetime, date
from pathlib import Path


In [ ]:
# | exporti
class Article(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    website_id: int = Field(foreign_key="website.id")
    file_path: str  # Path to .md file
    focus_keyword: str | None = None
    secondary_keywords: list[str] | None = Field(default=None, sa_column=Column(JSON))

    created_at: datetime = Field(default_factory=datetime.now)
    target_goal: str | None = None
    last_optimized: datetime | None = None


In [ ]:
# | export
def insert_article(
    session: Session,
    website_id: int,
    file_path: str,
    focus_keyword: str | None = None,
    secondary_keywords: list[str] | None = None,
    target_goal: str | None = None,
    last_optimized: datetime | None = None,
) -> Article:
    """Insert new article"""
    article = Article(
        website_id=website_id,
        file_path=file_path,
        focus_keyword=focus_keyword,
        secondary_keywords=secondary_keywords,
        target_goal=target_goal,
        last_optimized=last_optimized,
    )
    session.add(article)
    session.commit()
    session.refresh(article)
    return article


In [ ]:
# | export
def get_article_by_id(session: Session, article_id: int) -> Article | None:
    """Get article by ID"""
    return session.get(Article, article_id)


In [ ]:
# | export
def get_article_by_path(session: Session, file_path: str) -> Article | None:
    """Get article by file path"""
    statement = select(Article).where(Article.file_path == file_path)
    return session.exec(statement).first()


In [ ]:
# | export
def get_articles_by_website(session: Session, website_id: int) -> list[Article]:
    """Get all articles for a website"""
    statement = select(Article).where(Article.website_id == website_id)
    return session.exec(statement).all()


In [ ]:
# | export
def update_article_keyword(
    session: Session, article_id: int, focus_keyword: str
) -> Article | None:
    """Update article focus keyword"""
    article = session.get(Article, article_id)
    if article:
        article.focus_keyword = focus_keyword
        session.add(article)
        session.commit()
        session.refresh(article)
    return article


In [ ]:
# | export
def delete_article(session: Session, article_id: int) -> bool:
    """Delete article"""
    article = session.get(Article, article_id)
    if article:
        session.delete(article)
        session.commit()
        return True
    return False

In [ ]:
# | export
def update_article_optimization(
    session: Session,
    article_id: int,
    target_goal: str | None = None,
    focus_keyword: str | None = None,
    secondary_keywords: list[str] | None = None,
) -> Article | None:
    """Update article optimization fields and set last_optimized to now"""
    article = session.get(Article, article_id)
    if not article:
        return None

    if target_goal:
        article.target_goal = target_goal
    if focus_keyword:
        article.focus_keyword = focus_keyword
    if secondary_keywords:
        article.secondary_keywords = secondary_keywords

    article.last_optimized = datetime.now()

    session.add(article)
    session.commit()
    session.refresh(article)

    return article


In [ ]:
# | test
from fastcore.test import test_eq
from sqlmodel import create_engine, Session, SQLModel
from seo_rat.models import Website
from seo_rat.article import Article


In [ ]:
# | test

# Create in-memory database
engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    # Create website first
    website = Website(url="https://test.com", name="Test Site", lang="en")
    session.add(website)
    session.commit()
    session.refresh(website)

    # Test insert article
    article = insert_article(
        session, website_id=website.id, file_path="/test/article.md"
    )
    test_eq(article.file_path, "/test/article.md")
    # Test get by path
    found = get_article_by_path(session, "/test/article.md")
    test_eq(found.file_path, "/test/article.md")

    # Test update keyword
    updated = update_article_keyword(session, article.id, "new keyword")
    test_eq(updated.focus_keyword, "new keyword")

    # Test delete
    deleted = delete_article(session, article.id)
    test_eq(deleted, True)
    article2 = insert_article(
        session,
        website_id=website.id,
        file_path="/test/article.md",
        focus_keyword="new keyword",
        secondary_keywords=["keyword1", "keyword2"],
    )
    print(article2)

    test_eq(article2.secondary_keywords, ["keyword1", "keyword2"])
    optimized = update_article_optimization(
        session,
        article2.id,
        target_goal="rank for python bm25",
        focus_keyword="bm25 python",
    )
    test_eq(optimized.target_goal, "rank for python bm25")
    test_eq(optimized.last_optimized is not None, True)


website_id=1 id=1 secondary_keywords=['keyword1', 'keyword2'] created_at=datetime.datetime(2026, 2, 28, 9, 20, 3, 987508) last_optimized=None file_path='/test/article.md' focus_keyword='new keyword' target_goal=None
